#### Import all the libraries

In [3]:
import pandas as pd
import requests
import psycopg2
import time

#### Extract customer id and contact_email from the table customer

In [4]:

#lets create a function to connect to DB

def connect_to_postgres():
#lets declare all the connection parameters to the postgres DB
    db_params = {
                "host":'localhost',
                "database":'postgres',
                "user":'postgres',
                "password":'Your Server Password'
                }
    connection = psycopg2.connect(**db_params)
    return connection

# declare the name of your PostgreSQL table and columns
table_name = 'customers'
column_name1 = 'id'
colum_name2 = 'contact_email'

try: 
    connection = connect_to_postgres()
    # Construct the SQL query to select columns from the customer table
    query = f"SELECT {column_name1},{colum_name2} FROM {table_name}"

    # read the data into a DataFrame
    df = pd.read_sql_query(query, connection)

    # Close the database connection
    connection.close()
except psycopg2.Error as error:
    print("Error occured, Details:", error)
except Exception as error:
    print("Error occured, Details:", error)
finally:
    connection.close()

#create a copy of the df which will later used to merge after the indutry information is retrieved
df_backup = df.copy()

#  Display the DataFrame
# print(df)

#### Extract the domain name from email and remove duplicates to reduce API calls 

In [5]:
#change the data type of contact email to string
df['contact_email'] = df['contact_email'].astype(str)
#split the email from @ and get the domain
domain = df['contact_email'].str.split('@').str[1]
# remove the dupicates to reduce the API calls
domain.drop_duplicates(inplace=True)
#reset the index
domain = domain.reset_index(drop=True)
#change the domain to a df which will be later used to merge
df = pd.DataFrame(domain)
#create a new column industry to store the result from the API
df['industry'] = None
# print(df)

#### Function to fetch the industry from the Enrich API

In [6]:
def fetchIndustry(df,df_merge,batchno,batchsize):
    url = "https://api.apollo.io/v1/organizations/enrich"
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json'
    }
    #slice the df to smaller batch size, as 50 is allowed per minute
    df = df.loc[batchsize*batchno:(batchsize*(batchno+1)-1),]
    #iterate through the df, fetch the industry and store in the industry column
    for i in range(batchsize*batchno,batchsize*(batchno+1)):
        domain = df.loc[i,'contact_email'] 
        querystring = {
            "api_key": "Your API key",
            "domain": domain
        }
        try:
            response = requests.request("GET", url, headers=headers, params=querystring)
            if response.status_code == 200: #if request is okay 
                industry = response.json()
                if bool(response.json()) and bool(industry["organization"]["industry"]):#if the response is not empty and industry is not an empty string
                    # print(i)
                    df.loc[i,'industry'] = industry["organization"]["industry"]#save the industry to the df industry column
                else:
                   # print(i)
                    df.loc[i,'industry'] = None
            elif response.status_code == 429: #indicates that the API request weas exahusted
                print("API requests exhausted")
            else:
                print(response.text)
        except Exception as error:
        # Print the  error message
            print("An error occurred:", error)
    #save the df to csv file
    df_merge = pd.concat([df_merge, df])
    df_merge.to_csv('TestCustidandIndustryInfo.csv',header=1,index=0)
    return




#### Create a loop that runs every hour for 3 times and 4 times every 2 min as we have the API restriction (600 request per day, 200 req per hour and 50 request every min)

In [ ]:
batchsize = 20
# Run  every hour for 3 times
for i in range(3):
    if "df_merge" not in locals():
        df_merge = pd.DataFrame()
        # batchno = int(len(df_merge)/batchsize)
    else:
        df_merge = pd.read_csv('CustidandIndustryInfo.csv')
        # batchno = int(len(df_merge)/batchsize)
    # Run the program every 2 minutes for 4 times
    for j in range(4):
        batchno = int(len(df_merge)/batchsize)
        fetchIndustry(df,df_merge,batchno,batchsize) #run the function fetchIndustry
        df_merge = pd.read_csv('CustidandIndustryInfo.csv')
        time.sleep(120)  # Sleep for 2 minutes (120 seconds)
    time.sleep(3600)  # Sleep for 1 hour (3600 seconds)

#### Join the two df to have the industry for all the contact email's

In [7]:
#get the industry domain df
df_merge = pd.read_csv('CustidandIndustryInfo.csv')
#change the data type of contact email to string
df_backup['contact_email'] = df_backup['contact_email'].astype(str)
#split the email from @ and get the domain
df_backup['domain'] = df_backup['contact_email'].str.split('@').str[1]
#join with the df with so that it can be inserted in DB with correct cust id
df_insert = pd.merge(df_backup,df_merge,left_on='domain',right_on='contact_email',how="left")
# drop the unnecessary column 
df_insert = df_insert[['id','industry']]
#drop the duplicates
df_insert.drop_duplicates(inplace=True)
#replace the NaN value to null
df_insert = df_insert.where(pd.notnull(df_insert), None)
print(df_insert)

           id                           industry
0     1055178                               None
1     1037261                               None
2     1047267                               None
3     1050123                           printing
4     1097555                               None
...       ...                                ...
3730  1021042                               None
3731  1057037                               None
3732  1090115                               None
3733  1025342  information technology & services
3734  1036675                               None

[3701 rows x 2 columns]


#### Insert the industry value to Customer table 

In [8]:
try:
    connection = connect_to_postgres() # get the connection
    cursor = connection.cursor()

    # declare the table and column name
    table_name = 'customers'
    new_column_name = 'industry'
    data_type = 'VARCHAR(99)'  # Adjust the data type based on your requirements

    # Construct the SQL query to add the new column
    query = f"ALTER TABLE {table_name} ADD COLUMN {new_column_name} {data_type};"

    # Execute the query
    cursor.execute(query)

    # Use customer id to insert the industry 
    for index, row in df_insert.iterrows():
        primary_key_value = row['id']
        new_column_value = row['industry']

        # Construct the SQL query
        query = f"UPDATE {table_name} SET industry = %s WHERE id = %s"
        values = (new_column_value, primary_key_value)
        # Execute the query
        cursor.execute(query, values)

    # Commit the changes and close the connection
    connection.commit()
    
    connection.close()
except psycopg2.Error as error:
    print("Error occured, Details:", error)
except Exception as error:
    print("Error occured, Details:", error)
finally:
    cursor.close()
    connection.close()
